In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d minhhuy2810/rice-diseases-image-dataset

100% 12.0G/12.0G [04:03<00:00, 61.8MB/s]
100% 12.0G/12.0G [04:04<00:00, 52.7MB/s]


In [4]:
import zipfile

# Extract zip
local_zip = '/content/rice-diseases-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/rice-diseases-image-dataset')
zip_ref.close()

Import neccessary packages

In [44]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense, Input
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [71]:
EPOCHS = 100
LEARNING_RATE = 1e-3 * 3
BS = 32
DATA_DIR = os.path.join('/', 'content', 'rice-diseases-image-dataset')
LABELLED_DIR = os.path.join(DATA_DIR, 'LabelledRice', 'Labelled')
TRAINING_DIR = os.path.join(DATA_DIR, 'RiceDiseaseDataset', 'train')
VALIDATION_DIR = os.path.join(DATA_DIR, 'RiceDiseaseDataset', 'validation')
width = 256
height = 256
target_size = (width, height)
depth = 3

In [72]:
for subdir, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file == '.DS_Store':
            os.remove(os.path.join(subdir, file))

In [ ]:
# shuffle and split data
def split_data(data_dir, test_size=0.2):
    directories = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp'))]
        for f in file_names:
            images.append(f)
            labels.append(d)
    return train_test_split(images, labels, test_size=test_size, random_state=42)

resize_image = lambda img: tf.image.resize(img, target_size)

In [ ]:
# From a single image folder & shuffled
(train_images, train_labels), (test_images, test_labels) = split_data(LABELLED_DIR)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   )

val_datagen = ImageDataGenerator(rescale=1./255)

train_images = [resize_image(img) for img in train_images]
test_images = [resize_image(img) for img in test_images]

train_generator = train_datagen.flow(train_images, train_labels, batch_size=BS)
val_generator = val_datagen.flow(test_images, test_labels, batch_size=BS//2)

In [73]:
# # From A Single Image Folder
# datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=45,
#     width_shift_range=0.15,
#     height_shift_range=0.15,
#     shear_range=0.15,
#     zoom_range=0.2,
#     vertical_flip=True,
#     horizontal_flip=True,
#     validation_split=0.2,
# )

# train_generator = datagen.flow_from_directory(
#     LABELLED_DIR,
#     target_size=target_size,
#     batch_size=BS,
#     subset='training',
# )

# validation_generator = datagen.flow_from_directory(
#     LABELLED_DIR,
#     target_size=target_size,
#     batch_size=BS//2,
#     subset='validation',
# )

Found 2686 images belonging to 4 classes.
Found 669 images belonging to 4 classes.


In [74]:
# # From Separated Training & Validation Folder
# training_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=45,
#     width_shift_range=0.15,
#     height_shift_range=0.15,
#     shear_range=0.15,
#     zoom_range=0.2,
#     vertical_flip=True,
#     horizontal_flip=True,
# )

# validation_datagen = ImageDataGenerator(
#     rescale=1./255,
# )

# train_generator = training_datagen.flow_from_directory(
#     TRAINING_DIR,
#     target_size=target_size,
#     batch_size=BS,
# )

# validation_generator = validation_datagen.flow_from_directory(
#     VALIDATION_DIR,
#     target_size=target_size,
#     batch_size=BS,
# )

In [75]:
input_shape = target_size + (depth,)
chan_dim = -1
n_classes = train_generator.num_classes
if K.image_data_format() == 'channels_first':
    input_shape = (depth,) + target_size
    chan_dim = 1

In [76]:
# # Regular Learning
# model = Sequential([
#     Conv2D(32, (3, 3), padding="same",input_shape=input_shape),
#     Activation("relu"),
#     BatchNormalization(axis=chan_dim),
#     MaxPooling2D(pool_size=(3, 3)),
#     Dropout(0.25),
#     Conv2D(64, (3, 3), padding="same"),
#     Activation("relu"),
#     BatchNormalization(axis=chan_dim),
#     Conv2D(64, (3, 3), padding="same"),
#     Activation("relu"),
#     BatchNormalization(axis=chan_dim),
#     MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.25),
#     Conv2D(128, (3, 3), padding="same"),
#     Activation("relu"),
#     BatchNormalization(axis=chan_dim),
#     Conv2D(128, (3, 3), padding="same"),
#     Activation("relu"),
#     BatchNormalization(axis=chan_dim),
#     MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.25),
#     Flatten(),
#     Dense(1024),
#     Activation("relu"),
#     BatchNormalization(),
#     Dropout(0.5),
#     Dense(n_classes),
#     Activation("softmax")
# ])

In [77]:
# Transfer Learning
pre_trained_model = tf.keras.applications.DenseNet201(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    # pooling='avg',
)
pre_trained_model.trainable = False

In [78]:
pre_trained_model.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 262, 262, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 128, 128, 64  9408        ['zero_padding2d_6[0][0]']       
                                )                                                       

In [79]:
# # No additional Input layer
# inputs = pre_trained_model.input
# x = pre_trained_model(inputs, training=False)
# x = Dense(1024, activation=tf.nn.relu)(x)
# x = Dropout(0.2)(x)
# outputs = Dense(n_classes, activation=tf.nn.softmax)(x)
# model = Model(inputs=inputs, outputs=outputs)
# model.summary()

In [80]:
# Sequential
model = Sequential([
  pre_trained_model,
  # Dense(1024, activation=tf.nn.relu),
  Conv2D(512, 1, padding='same', activation=tf.nn.relu),
  MaxPooling2D(2, 1, padding='same'),
  Conv2D(256, 1, padding='same', activation=tf.nn.relu),
  GlobalAveragePooling2D(),
  BatchNormalization(),
  Dense(128, activation=tf.nn.relu),
  BatchNormalization(),
  Dropout(0.4),
  Dense(n_classes, activation=tf.nn.softmax)
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 8, 8, 1920)        18321984  
                                                                 
 conv2d_12 (Conv2D)          (None, 8, 8, 512)         983552    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 512)        0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 8, 8, 256)         131328    
                                                                 
 global_average_pooling2d_3   (None, 256)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_6 (Batc  (None, 256)             

In [81]:
# optimizer = Adam(learning_rate=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [82]:
BEST_MODEL = 'saved_model/best_model'
BEST_MODEL_H5 = BEST_MODEL + '.h5'

def get_model_checkpoint(metrics_values={}):
    evaluated_metrics = lambda logs: [logs[key] > val if 'accuracy' in key else logs[key] < val for key, val in metrics_values.items()]

    class MyModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
        def on_epoch_end(self, epoch, logs):
            if False not in evaluated_metrics(logs):
                super().on_epoch_end(epoch, logs)
                global best_model
                best_model = self.model
    monitor = 'val_loss' if not metrics_values.keys() else next(iter(metrics_values.keys()))
    return MyModelCheckpoint(
        BEST_MODEL_H5,
        verbose=1,
        save_best_only=True,
        monitor=monitor)


In [83]:
model_checkpoint = get_model_checkpoint({'val_accuracy': 0.7})

In [84]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(factor=0.3,
                                                               patience=5,
                                                               min_lr=1e-6)

In [85]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=(EPOCHS*1e-1)//1)

In [86]:
# learning_rate_schedule = tf.keras.callbacks.LearningRateScheduler(
#     lambda epoch: LEARNING_RATE * 10**(epoch/2)
# )

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[model_checkpoint, learning_rate_reduction, early_stopping],
    # callbacks=[learning_rate_schedule],
)

Epoch 1/100
84/84 [==============================] - 401s 5s/step - loss: 1.2542 - accuracy: 0.5491 - val_loss: 2.6282 - val_accuracy: 0.3004 - lr: 0.0030
Epoch 2/100
84/84 [==============================] - 388s 5s/step - loss: 0.9431 - accuracy: 0.6359 - val_loss: 1.4590 - val_accuracy: 0.4753 - lr: 0.0030
Epoch 3/100
84/84 [==============================] - 391s 5s/step - loss: 0.8590 - accuracy: 0.6582 - val_loss: 1.3527 - val_accuracy: 0.5725 - lr: 0.0030
Epoch 4/100
30/84 [=========>....................] - ETA: 3:10 - loss: 0.8700 - accuracy: 0.6500

Plot the train and val curve

In [ ]:
# plt.semilogx(history.history['lr'], history.history['loss'])
# plt.axis([LEARNING_RATE, 1e-0, 0, 1.5])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()

#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate(validation_generator)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model(BEST_MODEL_H5)
scores = best_model.evaluate(validation_generator)
print(f"Best Accuracy: {scores[1]*100}")

In [ ]:
best_model.save('/content/saved_model/model')
!zip -r /content/model.zip /content/saved_model/model